# Test models

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3,5"

import torch
import orbax
from pathlib import Path
from typing import Any, Dict
from flax.training import orbax_utils
from flax.training.train_state import TrainState

In [2]:
def load_tokenizer(ckpt_path: str):
    orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
    restored = orbax_checkpointer.restore(ckpt_path)
    return restored["model"]

def load_dynamics(ckpt_path: str):
    orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
    restored = orbax_checkpointer.restore(ckpt_path)
    return restored["model"]

def load_rew_end_model(pt_path: str, device: str = "cuda"):
    from models.rew_end_model import RewEndModel, RewEndModelConfig

    model = RewEndModel(RewEndModelConfig(
        lstm_dim=512,
        img_channels=3,
        img_size=64,
        cond_channels=128,
        depths=[2, 2, 2],
        channels=[64, 128, 256],
        attn_depths=[0, 0, 1],
        num_actions=15,
    ))
    checkpoint = torch.load(pt_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval().to(device)
    return model

In [4]:
tokenizer = load_tokenizer('/homes/53/fpinto/jafar/video_tokeniser_ckpt/tokenizer_1749152837_200000')

In [5]:
dynamics = load_dynamics('/homes/53/fpinto/jafar/dynamics_ckpt/genie_1749549031_25000')

In [6]:
rew_end = load_rew_end_model('/homes/53/fpinto/jafar/rew_end_ckpt/rew_end_ckpt_0999.pt')

OutOfMemoryError: CUDA out of memory. Tried to allocate 518.00 MiB (GPU 0; 44.40 GiB total capacity; 26.91 MiB already allocated; 89.00 MiB free; 32.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF